# Lime: Explaining predictions 

### What is it ?

Local Interpretable Model-Agnostic Explanations (LIME) is a technique for explaining the decisions made by a machine learning model. The designers of LIME state a goal of identifying an interpretable model over the interpretable representation that is locally faithful to the classifier. The authors define their concept of interpretable representations as human understandable analogs for features used in real world models. With the LIME algorithm the authors hope to “explain the predictions of any classifier or regressor in a faithful way, by approximating it locally with an interpretable model”.  This means that using LIME a simple model that is easy to understand is used to explain the predictions of a more complex model in a localized region. 

## How to use lime 

### Instalation 

LIME is disttributed as a pyhton package instalable with pip or can be downloaded directly from the projects [repository](https://github.com/marcotcr/lime) current versions of the library only support python 3 

~~~ bash
pip install lime 
~~~

### Basic Usage 

LIME has methods for explaining many different types of Model for this basic tutorial we will use the tool to explain the predictions of a random forest classifier. first we have to import LIME in the normal way 

~~~ python
import lime 

~~~

The first step in using LIME is initialising the LimeTextExplainer class with the class_names variable for the different classes the explainer will be identifying. LIME has many explainer methods that can be used to explain different models 


~~~ python 
lime_explainer = lime_text.LimeTextExplainer(class_names=["positive","negative"])
~~~

Once the Explainer is intialized it can be used to explain a prediction made by a model. For some models the data is pre-processed into vectorized sets (this is the case for some sklearn models) in these cases we may need to set up a pipline to get access to the unprocessed text data. The max number of features also needs to be apllied to the explainer.


~~~ python 
x = 42 # row of data to be classified and explained 
explaination = explainer.explain_instance(data[x], classfier, num_features=6)
~~~

The `explaination` object returned by `explain_instance` contains a linnear aproximation of the classifier for the provided data row. This can be used in a number of ways to comunicate the explaination to the user 


### Types of Explainer

As mentioned above there are several different LIME explainer classes used for different types of data. We have already seen `lime_text` , but the class also includes `lime_tabular` and `lime_images` for tabular and image data respectivly. 


###  `lime_images`

We can seperatly import the diferent lime components using the `from` import pattern. Here we import the lime images component.We will use an example from the lime documentation.

~~~ python 
from lime import lime_image
~~~

For the image explainer it is not nessasary to initialize the explainer with lables like the text explainer 

~~~ python 
explainer = lime_image.LimeImageExplainer()
~~~

Building an explaination for an image instance requires different parameters than the explainer for text models.
This example taken from the LIME documentation is explaining a model which idetifies cats and dogs in an image.
we can see that for this explainer we need to pass the image and the model in addition to the `top_labels` parameter which controls the number of labels the explainer will return. `hide_color` which is the colour for a super pixel to be disabled, this value can also be `None`. `num_samples` is the size of the neighborhood to learn the linear model.

~~~ python 
explanation = explainer.explain_instance(image, predict_fn, top_labels=5, hide_color=0, num_samples=1000)
~~~

Like the explain instance for `lime_text` this instance contains a linear model of the local being explained. However, the explaination produced has different methods for displaying the explaination includig mathods for masking and superimposing colour over the classified image. 


###  `lime_tabular`

The `lime_tabular` explainer is used to explain predictions on matrix data. This explainer differs from the `lime_text` and `lime_images` explainers in that it requires a training set in order to instantiate the explainer. we will use an example from the LIME documentation to demonstarte this class. 
first we import the class in the usual way.

~~~ python 
from lime import lime_tabular
~~~

Then we instatniate the explainer 

~~~ python
explainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=feature_names, class_names=target_names, discretize_continuous=True)
~~~


finally we create the explaination 

~~~ python 
explaination = explainer.explain_instance(test[x], model, num_features=2, top_labels=1)
~~~
### Explaining Instances 


In [5]:
#load data 
import pandas as pd
import numpy as np
import requests 
import tarfile
import os 
import gensim
import gensim.downloader
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

url = "https://www.cs.jhu.edu/~mdredze/datasets/sentiment/processed_acl.tar.gz"
filename = "processed_acl"
extension = ".tar.gz"

response = requests.get(url, allow_redirects=True)

print("http response:",response.status_code,response.reason)

with open(filename+extension,"wb") as file: 
    file.write(response.content)
    
tar = tarfile.open(filename+extension, "r:gz")
tar.extractall()
tar.close()
# # merge files using shell commands 
print(os.popen("cat " + filename+"/books/negative.review > mixed.txt").read())
print(os.popen("cat " + filename+"/books/positive.review >> mixed.txt").read())
# print(os.popen("cat " + filename+"/dvd/all.review >> mixed.txt").read())
# print(os.popen("cat " + filename+"/dvd/positive.review >> mixed.txt").read())



# parse data into array for word 2 vect 
dataArray = []
labelArray = []

with open("mixed.txt", "r") as datafile: 
    lines = datafile.readlines()
    for e in lines:
        lineList= []
        splitLine = e.split()
        label = splitLine.pop(-1)
        for i in splitLine:
            dictSplit = i.split(':')
            lineList.append(dictSplit[0])
        dataArray.insert(-1, lineList)
        labelArray.insert(-1, (label.split(':')[1]))

#download word2vec model          
word2vec = gensim.downloader.load('fasttext-wiki-news-subwords-300')


# create mean embedding 
embed = np.array([ np.mean([word2vec[w] for w in words if w in word2vec], axis=0) for words in dataArray])

X_train, X_test, y_train, y_test = train_test_split(embed, labelArray, test_size=0.2, random_state=42)

text_classifier = RandomForestClassifier(n_estimators=700, random_state=42)
text_classifier.fit(X_train, y_train)

predictions = text_classifier.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))



http response: 200 OK


[[164  35]
 [ 37 164]]
              precision    recall  f1-score   support

    negative       0.82      0.82      0.82       199
    positive       0.82      0.82      0.82       201

    accuracy                           0.82       400
   macro avg       0.82      0.82      0.82       400
weighted avg       0.82      0.82      0.82       400

0.82


In [ ]:
#  dl tubspam data set 
# load into numpy array 
# create embedings on full text 
# use classifier to classify the dataset 


In [ ]:
#Lime explainer 